In [1]:
import pandas as pd
import spacy
#!python -m spacy download ru_core_news_md;
import ru_core_news_md;
nlp = ru_core_news_md.load()

In [38]:
class Dialog_parser:
    GREETING = ['здравствуйте', 'приветствую', 'добрый день', 'доброе утро', 'добрый вечер']
    FAREWELL = ['всего хорошего', 'до свидания', 'всего доброго', 'хорошего вам дня', 'хорошего дня','хорошего вечера', 'до встречи', 'до связи', 'будьте здоровы']

    def __init__(self, filename) -> None:
        self.filename=filename
        data = pd.read_csv(self.filename)
        data = data[data['role'] == 'manager'].reset_index()

    @classmethod
    def has_greeting(cls, str):
        return any([greeting in str.lower() for greeting in cls.GREETING])

    @classmethod
    def has_farewell(cls, str):
        return any([farewell in str.lower() for farewell in cls.FAREWELL])

    @staticmethod
    def get_names(str):
        names = []
        for word in str.split():
            doc =  nlp(word.title())
            for ent in doc.ents:
                if ent.label_ == 'PER':
                    names.append(ent.text)
        return names if names else 0

    def get_greeting(self):
        result = []

        data['greeting'] = data['text'].apply(self.has_greeting)
        ids = set(data[(data['greeting'] == True)]['dlg_id'])
        for id in ids:
            text = (data[(data['dlg_id'] == id) & (data['greeting'] == True)]['text']).to_string(index=False)
            result.append(f'dlg_id={id}: {text}')
        return result

    def get_introduction(self):
        result = []

        data['PER'] = data['text'].apply(self.get_names)
        df = data[(data['PER'] != 0) & (data['text'].str.contains(('|').join(['это','зовут', 'говорит', 'представиться'])))]
        ids = set(df['dlg_id'])
        for id in ids:
            text = df[df['dlg_id'] == id]['text'].iloc[0]
            name = df[df['dlg_id'] == id]['PER'].iloc[0][0].title()
            result.append(f'dlg_id={id}. Имя: {name}. Реплика: {text}')    
        return result
    
    def get_company(self):
        result = []
        df = data[(data['text'].str.contains('компания'))]
        for id in set(df['dlg_id']):
            words = (df[df['dlg_id'] == id]['text'].iloc[0]).split()
            company = words[words.index('компания') + 1]
            result.append(f'dlg_id={id}. Компания: {company}')
        return result
    
    def get_farewell(self):
        result = []
        data['farewell'] = data['text'].apply(self.has_farewell)
        ids = set(data[(data['farewell'] == True)]['dlg_id'])
        for id in ids:
            text = (data[(data['dlg_id'] == id) & (data['farewell'] == True)]['text'].iloc[0])
            result.append(f'dlg_id={id}: {text}')
        return result

    def check_manager(self):
        result = []
        for id in data['dlg_id'].unique():
            hello = f'dlg_id={id}' in ''.join(self.get_greeting())
            goodbuy = f'dlg_id={id}' in ''.join(self.get_farewell())
            result.append(f'dlg_id={id}. Вежливость менеджера: {hello & goodbuy}')
        return result


In [39]:
parser = Dialog_parser('test_data.csv')

In [ ]:
parser = Dialog_parser('test_data.csv')

In [40]:
parser.get_greeting()

['dlg_id=0: Алло здравствуйте',
 'dlg_id=1: Алло здравствуйте',
 'dlg_id=2: Алло здравствуйте',
 'dlg_id=3: Алло дмитрий добрый день']

In [41]:
parser.get_introduction()

['dlg_id=0. Имя: Ангелина. Реплика: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
 'dlg_id=1. Имя: Ангелина. Реплика: Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
 'dlg_id=2. Имя: Ангелина. Реплика: Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
 'dlg_id=3. Имя: Максим. Реплика: Добрый меня максим зовут компания китобизнес удобно говорить',
 'dlg_id=5. Имя: Анастасия. Реплика: Да это анастасия']

In [42]:
parser.get_company()

['dlg_id=0. Компания: диджитал',
 'dlg_id=1. Компания: диджитал',
 'dlg_id=2. Компания: диджитал',
 'dlg_id=3. Компания: китобизнес']

In [43]:
parser.get_farewell()

['dlg_id=0: Всего хорошего до свидания',
 'dlg_id=1: Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
 'dlg_id=3: Угу все хорошо да понедельника тогда всего доброго',
 'dlg_id=4: Во вторник все ну с вами да тогда до вторника до свидания',
 'dlg_id=5: Ну до свидания хорошего вечера']

In [44]:
parser.check_manager()

['dlg_id=0. Вежливость менеджера: True',
 'dlg_id=1. Вежливость менеджера: True',
 'dlg_id=2. Вежливость менеджера: False',
 'dlg_id=3. Вежливость менеджера: True',
 'dlg_id=4. Вежливость менеджера: False',
 'dlg_id=5. Вежливость менеджера: False']